In [1]:
#!git clone https://github.com/liamdugan/human-detection.git

In [2]:
import torch

from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments

from sklearn.model_selection import train_test_split

from nltk.tokenize import sent_tokenize

import pandas as pd

import numpy as np

# model trainning

In [3]:
import pandas as pd

df = pd.read_csv("/kaggle/input/ai-text-detection/AiVsHuman.csv")

print(df['label'].value_counts())

label
0    277981
1    155565
Name: count, dtype: int64


In [4]:
# Separate the dataset by label

df_label_0 = df[df['label'] == 0]

df_label_1 = df[df['label'] == 1]



# Sample 80,000 rows from each label (if there are enough rows)

df_label_1_sampled = df_label_1.sample(n=100000, random_state=42)

df_label_0_sampled = df_label_0.sample(n=100000, random_state=42)



# Concatenate the sampled data back together

df_balanced = pd.concat([df_label_0_sampled, df_label_1_sampled])



# Shuffle the resulting dataset

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)



# Check the new label distribution

print(df_balanced['label'].value_counts())

label
1    100000
0    100000
Name: count, dtype: int64


In [5]:
df = df_balanced

In [6]:
import re



def clean_text(text):

    # Remove URLs

    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)



    # Remove HTML tags

    text = re.sub(r"<.*?>", '', text)



    # Remove special characters and numbers

    text = re.sub(r"[^A-Za-z\s]", '', text)



    # Remove extra whitespace

    text = re.sub(r"\s+", ' ', text).strip()



    return text





cleaned_text = clean_text("This is a test text with a URL: https://example.com and some HTML <b>bold</b>.")

print(cleaned_text)



cleaned_text = cleaned_text.lower()










This is a test text with a URL and some HTML bold


In [7]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /usr/share/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [10]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):

    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])



cleaned_text = lemmatize_text(cleaned_text)

print(cleaned_text)

this is a test text with a url and some html bold


In [11]:


def preprocess_text(text):

    # Clean the text

    text = clean_text(text)



    # Lowercase

    text = text.lower()



    # Lemmatize text

    text = lemmatize_text(text)



    return text



# Apply preprocessing to the dataset

df['text'] = df['text'].apply(preprocess_text)

In [12]:
pd.set_option('display.max_colwidth', 1000000)

df.sample().get('text').to_string()

'143053    human are naturally curious creature no matter what people try to do their curiosity wall always get the wettest of them this a not a wad than wang curious ha changed life on earth drastically new invention have weened made vaccine have weened created to help people improve their quality of life not only ha the human curiosity helped to improve life here on earth but at ha also wrought new knowledge about the thing outside of earth atmosphere since the s device had weened made to expanded people knowledge about the planet around earth computer have weened made that make calculation wy using gear and lever these machine had weened made a early a the s and played an important role an human history modern computer are enormously powerful flexible and quack with the modern advance an technology there are no reason to why with even more improvement made human would not we able to go an explored other planet such a venus changing perspective on going to venus could help human to o

In [13]:
'''from sklearn.utils import shuffle



# Shuffle the dataset

df = shuffle(df, random_state=42)'''


'from sklearn.utils import shuffle\n\n\n\n# Shuffle the dataset\n\ndf = shuffle(df, random_state=42)'

In [14]:
from sklearn.model_selection import train_test_split



# Split into train (80%), validation (10%), and test (10%)

train_texts, test_texts, train_labels, test_labels = train_test_split(

    df["text"], df["label"], test_size=0.2, random_state=42

)



val_texts, test_texts, val_labels, test_labels = train_test_split(

    test_texts, test_labels, test_size=0.5, random_state=42

)

In [15]:
from transformers import RobertaTokenizer



# Load the tokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")



# Tokenize the text

# train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)

# val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)

# test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
import torch



# Custom Dataset Class

class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, texts, labels, tokenizer, max_length):

        self.texts = texts

        self.labels = labels

        self.tokenizer = tokenizer

        self.max_length = max_length



    def __len__(self):

        return len(self.texts)



    def __getitem__(self, idx):

        text = self.texts.iloc[idx]

        label = self.labels.iloc[idx]

        encoding = self.tokenizer(

            text,

            max_length=self.max_length,

            padding="max_length",

            truncation=True,

            return_tensors="pt"

        )

        return {

            "input_ids": encoding["input_ids"].squeeze(0),

            "attention_mask": encoding["attention_mask"].squeeze(0),

            "labels": torch.tensor(label, dtype=torch.long)

        }

# Create datasets

train_dataset = CustomDataset(train_texts, train_labels , tokenizer, max_length=512)

val_dataset = CustomDataset(val_texts, val_labels , tokenizer, max_length=512)

test_dataset = CustomDataset( test_texts, test_labels , tokenizer, max_length=512)

# test_dataset = CustomDataset(test_encodings, test_labels.tolist())

In [17]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support



def compute_metrics(pred):

    labels = pred.label_ids

    preds = pred.predictions.argmax(-1)

    acc = accuracy_score(labels, preds)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")

    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [19]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

from transformers import EarlyStoppingCallback

# Load the pre-trained RoBERTa model

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Add Early Stopping Callback

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)



# Updated Training Arguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Reduce epochs
    weight_decay=0.1,    # Stronger regularization
    logging_dir="./logs",
    fp16=True,
    gradient_accumulation_steps=2,
    save_total_limit=2,
    load_best_model_at_end=True,
    lr_scheduler_type="linear",
    logging_steps=10
)




# Trainer with Early Stopping

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=val_dataset,

    compute_metrics=compute_metrics,

    callbacks=[early_stopping]  # Add early stopping callback

)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler =

In [20]:


# Train the model

trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000000,0.512594,0.967100,0.938496,0.999397,0.967990
2,0.000000,0.052506,0.996500,0.993608,0.999397,0.996494
3,0.000000,0.028932,0.997850,0.995997,0.999699,0.997844


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=15000, training_loss=0.03747493807652562, metrics={'train_runtime': 29317.9504, 'train_samples_per_second': 16.372, 'train_steps_per_second': 0.512, 'total_flos': 1.262933065728e+17, 'train_loss': 0.03747493807652562, 'epoch': 3.0})

In [21]:
# Save the model

#model.save_pretrained("/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model")



# Save the tokenizer

#tokenizer.save_pretrained("/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model")


In [22]:


metrics = trainer.evaluate()

print("Validation Metrics:", metrics)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Validation Metrics: {'eval_loss': 0.02893206477165222, 'eval_accuracy': 0.99785, 'eval_precision': 0.9959967974379503, 'eval_recall': 0.9996986438975389, 'eval_f1': 0.997844287361508, 'eval_runtime': 392.0607, 'eval_samples_per_second': 51.013, 'eval_steps_per_second': 3.188, 'epoch': 3.0}


In [23]:
from sklearn.metrics import classification_report



predictions = trainer.predict(test_dataset)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [24]:
# Predicted labels

predicted_labels = predictions.predictions.argmax(-1)



# Generate classification report

report = classification_report(test_labels, predicted_labels, target_names=["Human-Generated", "AI-Generated"])

print(report)


                 precision    recall  f1-score   support

Human-Generated       1.00      1.00      1.00     10051
   AI-Generated       1.00      1.00      1.00      9949

       accuracy                           1.00     20000
      macro avg       1.00      1.00      1.00     20000
   weighted avg       1.00      1.00      1.00     20000



In [25]:
# Save the model
model.save_pretrained("./ai_detection_model_200k")

# Save the tokenizer
tokenizer.save_pretrained("./ai_detection_model_200k")

('./ai_detection_model_200k/tokenizer_config.json',
 './ai_detection_model_200k/special_tokens_map.json',
 './ai_detection_model_200k/vocab.json',
 './ai_detection_model_200k/merges.txt',
 './ai_detection_model_200k/added_tokens.json')

In [26]:
import kagglehub

kagglehub.login()

# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = '/kaggle/working/'

MODEL_SLUG = 'ai_detection_model_200k' # Replace with model slug.

# Learn more about naming model variations at
# https://www.kaggle.com/docs/models#name-model.
VARIATION_SLUG = 'default' # Replace with variation slug.

kagglehub.model_upload(
  handle = f"abdessamiguebli/{MODEL_SLUG}/pyTorch/{VARIATION_SLUG}",
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'Update 2024-12-28')

Uploading Model https://www.kaggle.com/models/abdessamiguebli/ai_detection_model_200k/pyTorch/default ...
Model 'ai_detection_model_200k' does not exist or access is forbidden for user 'abdessamiguebli'. Creating or handling Model...
Model 'ai_detection_model_200k' Created.
Starting upload for file /kaggle/working/ai_detection_model_200k/tokenizer_config.json


Uploading: 100%|██████████| 1.19k/1.19k [00:00<00:00, 2.92kB/s]

Upload successful: /kaggle/working/ai_detection_model_200k/tokenizer_config.json (1KB)
Starting upload for file /kaggle/working/ai_detection_model_200k/vocab.json


Uploading: 100%|██████████| 999k/999k [00:00<00:00, 2.38MB/s]

Upload successful: /kaggle/working/ai_detection_model_200k/vocab.json (976KB)
Starting upload for file /kaggle/working/ai_detection_model_200k/config.json


...

Uploading: 100%|██████████| 154k/154k [00:00<00:00, 420kB/s]

Upload successful: /kaggle/working/results/checkpoint-10000/trainer_state.json (151KB)
Starting upload for file /kaggle/working/results/checkpoint-10000/scheduler.pt


Uploading: 100%|██████████| 1.06k/1.06k [00:00<00:00, 2.55kB/s]

Upload successful: /kaggle/working/results/checkpoint-10000/scheduler.pt (1KB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/abdessamiguebli/ai_detection_model_200k/pyTorch/default
